In [1]:
import sys
sys.path.insert(0, '../../../fastshap_tf/')
from fastshap import ImageFastSHAP, ShapleySampler, ResizeMask

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import pickle
import numpy as np
import shap
from tqdm.notebook import tqdm
import time

In [3]:
import tensorflow as tf
from tensorflow.keras.layers import (Input, Layer, Dense, Lambda, Reshape, Multiply)
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers.experimental.preprocessing import Resizing

from tensorflow.keras.datasets import cifar10

from datetime import datetime
import os

In [4]:
#Select GPU
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [5]:
# IMPORTANT: SET RANDOM SEEDS FOR REPRODUCIBILITY
os.environ['PYTHONHASHSEED'] = str(420)
import random
random.seed(420)
np.random.seed(420)
tf.random.set_seed(420)

## Load Data

In [6]:
BATCH_SIZE = 32
EPOCHS = 100
LR = 1e-3
INPUT_SHAPE = (224, 224, 3)

In [7]:
from sklearn.model_selection import train_test_split

num_classes = 10

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test, train_size=0.5, random_state=420)

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')
#Resize to 224x224

print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'val samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_val = tf.keras.utils.to_categorical(y_val, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# Make TF Dataset
ds_train = tf.data.Dataset.from_tensor_slices((x_train, y_train))
ds_val = tf.data.Dataset.from_tensor_slices((x_val, y_val))
ds_test = tf.data.Dataset.from_tensor_slices((x_test, y_test))

50000 train samples
5000 val samples
5000 test samples


### Batch Data

In [8]:
def batch_data(dataset, fn, batch_size=32):
    dataset = dataset.map(fn)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    
    return dataset

### Reformat Data

In [9]:
def reformat(x, y):
    
    x = tf.cast(x, tf.float32)
    x = Resizing(INPUT_SHAPE[0], INPUT_SHAPE[1], interpolation='nearest')(x)
    x = tf.keras.applications.resnet50.preprocess_input(x)
    
    return (x, y)

ds_train = batch_data(ds_train, reformat, BATCH_SIZE)
ds_val = batch_data(ds_val, reformat, BATCH_SIZE)
ds_test = batch_data(ds_test, reformat, BATCH_SIZE)

## Load Surrogate Imputer

In [10]:
from tensorflow.keras.applications.resnet50 import ResNet50

P = 14*14
value_model = ResNet50(
    include_top=False, weights='imagenet', 
    input_shape=INPUT_SHAPE, pooling='avg'
) 
D = 10

model_input = Input(shape=INPUT_SHAPE, dtype='float64', name='input')
S = ShapleySampler(P, paired_sampling=False, num_samples=1)(model_input)
S = Lambda(lambda x: tf.cast(x, tf.float32))(S)
S = Reshape((P,))(S)
S = ResizeMask(in_shape=INPUT_SHAPE, mask_size=P)(S)
xs = Multiply()([model_input, S])

net = value_model(xs)
out = Dense(D, activation='softmax')(net)

surrogate = Model(model_input, out)

# Get Checkpointed Model
weights_path = 'surrogate/20210515_01_11_10/value_weights.h5'
surrogate.load_weights(weights_path)

# Remove Masking Layer
surrogate = Sequential(   
    [l for l in surrogate.layers[-2:]]
)
surrogate.trainable = False


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



# Train FastSHAP

### Save Dir

In [11]:
date = datetime.now().strftime("%Y%m%d_%H_%M_%S")
save_dir = 'fastshap'
model_dir = os.path.join(os.getcwd(), save_dir, date)
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

### Initialize

In [12]:
from importlib import reload
import fastshap
import utils
reload(fastshap)
reload(utils)
from fastshap import ImageFastSHAP

In [13]:
fastshap = ImageFastSHAP(imputer = surrogate,
                         normalization=None,
                         model_dir = model_dir, 
                         link='logit')

### Train

In [14]:
t = time.time()
fastshap.train(train_data = ds_train, 
              val_data = ds_val, 
              max_epochs = EPOCHS, 
              batch_size = BATCH_SIZE, 
              num_samples = 1,
              lr = LR,
              paired_sampling = True, 
              eff_lambda = 0.0,
              verbose = 1,
              lookback = 20)
training_time = time.time() - t

with open(os.path.join(model_dir, 'training_time.pkl'), 'wb') as f:
    pickle.dump(training_time, f)

The following Variables were used a Lambda layer's call (lambda_1), but
are not present in its tracked objects:
  <tf.Variable 'conv1_conv/kernel:0' shape=(7, 7, 3, 64) dtype=float32>
  <tf.Variable 'conv1_conv/bias:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv1_bn/gamma:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv1_bn/beta:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv2_block1_0_conv/kernel:0' shape=(1, 1, 64, 256) dtype=float32>
  <tf.Variable 'conv2_block1_0_conv/bias:0' shape=(256,) dtype=float32>
  <tf.Variable 'conv2_block1_0_bn/gamma:0' shape=(256,) dtype=float32>
  <tf.Variable 'conv2_block1_0_bn/beta:0' shape=(256,) dtype=float32>
  <tf.Variable 'conv2_block1_1_conv/kernel:0' shape=(1, 1, 64, 64) dtype=float32>
  <tf.Variable 'conv2_block1_1_conv/bias:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv2_block1_1_bn/gamma:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv2_block1_1_bn/beta:0' shape=(64,) dtype=float32>
  <tf.Variable 'conv2_block1_2_conv/kernel:0'

Epoch 1/100
1563/1563 [==============================] - ETA: 0s - loss: 60.2409 - shap_loss: 60.2409 - eff_loss: 0.0000e+00
Epoch 00001: val_shap_loss improved from inf to 25.81140, saving model to /gpfs/data/paulab/nj594/fast_shap/experiments/images/cifar10/fastshap/20210519_16_07_53/explainer_weights.h5
1563/1563 [==============================] - 361s 231ms/step - loss: 60.2409 - shap_loss: 60.2409 - eff_loss: 0.0000e+00 - val_loss: 25.8114 - val_shap_loss: 25.8114 - val_eff_loss: 0.0000e+00
Epoch 2/100
1563/1563 [==============================] - ETA: 0s - loss: 25.8413 - shap_loss: 25.8413 - eff_loss: 0.0000e+00
Epoch 00002: val_shap_loss improved from 25.81140 to 25.06451, saving model to /gpfs/data/paulab/nj594/fast_shap/experiments/images/cifar10/fastshap/20210519_16_07_53/explainer_weights.h5
1563/1563 [==============================] - 267s 171ms/step - loss: 25.8413 - shap_loss: 25.8413 - eff_loss: 0.0000e+00 - val_loss: 25.0645 - val_shap_loss: 25.0645 - val_eff_loss: 0.00

Epoch 19/100
1563/1563 [==============================] - ETA: 0s - loss: 16.0372 - shap_loss: 16.0372 - eff_loss: 0.0000e+00
Epoch 00019: val_shap_loss did not improve from 16.91743
1563/1563 [==============================] - 260s 167ms/step - loss: 16.0372 - shap_loss: 16.0372 - eff_loss: 0.0000e+00 - val_loss: 18.3632 - val_shap_loss: 18.3632 - val_eff_loss: 0.0000e+00
Epoch 20/100
1563/1563 [==============================] - ETA: 0s - loss: 15.8908 - shap_loss: 15.8908 - eff_loss: 0.0000e+00
Epoch 00020: val_shap_loss did not improve from 16.91743
1563/1563 [==============================] - 260s 167ms/step - loss: 15.8908 - shap_loss: 15.8908 - eff_loss: 0.0000e+00 - val_loss: 18.1049 - val_shap_loss: 18.1049 - val_eff_loss: 0.0000e+00
Epoch 21/100
1563/1563 [==============================] - ETA: 0s - loss: 15.6986 - shap_loss: 15.6986 - eff_loss: 0.0000e+00
Epoch 00021: val_shap_loss did not improve from 16.91743

Epoch 00021: ReduceLROnPlateau reducing learning rate to 0.00080

1563/1563 [==============================] - ETA: 0s - loss: 14.0302 - shap_loss: 14.0302 - eff_loss: 0.0000e+00
Epoch 00058: val_shap_loss did not improve from 15.50072

Epoch 00058: ReduceLROnPlateau reducing learning rate to 0.00016777217388153076.
1563/1563 [==============================] - 260s 167ms/step - loss: 14.0302 - shap_loss: 14.0302 - eff_loss: 0.0000e+00 - val_loss: 15.8949 - val_shap_loss: 15.8949 - val_eff_loss: 0.0000e+00
Epoch 59/100
1563/1563 [==============================] - ETA: 0s - loss: 13.9776 - shap_loss: 13.9776 - eff_loss: 0.0000e+00
Epoch 00059: val_shap_loss did not improve from 15.50072
1563/1563 [==============================] - 260s 167ms/step - loss: 13.9776 - shap_loss: 13.9776 - eff_loss: 0.0000e+00 - val_loss: 15.6759 - val_shap_loss: 15.6759 - val_eff_loss: 0.0000e+00
Epoch 60/100
1563/1563 [==============================] - ETA: 0s - loss: 13.9697 - shap_loss: 13.9697 - eff_loss: 0.0000e+00
Epoch 00060: val_shap_loss did not improve from 15.50

# Explain w/ FastSHAP

### Load Images

In [15]:
images_dir = os.path.join(os.getcwd(), 'images')
images = np.load(os.path.join(images_dir, 'processed_images.npy'), allow_pickle=True)

### Explain

In [18]:
t = time.time()
shap_values = fastshap.explainer.predict(images)
explaining_time = time.time() - t
shap_values = [shap_values[:,:,:,i] for i in range(10)]

### Save

In [19]:
# save_dir = 'fastshap'
# model_dir = os.path.join(os.getcwd(), save_dir, 'results')
if not os.path.isdir(model_dir):
    os.makedirs(model_dir)

with open(os.path.join(model_dir, 'explaining_time.pkl'), 'wb') as f:
    pickle.dump(explaining_time, f)
    
with open(os.path.join(model_dir, 'shap_values.pkl'), 'wb') as f:
    pickle.dump(shap_values, f)